# ALUMNO: Ezequiel Scordamaglia

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sistema de obtención de información con NLTK utilizando un corpus de wikipedia


In [111]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
# nltk.download()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EZEQUIEL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\EZEQUIEL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\EZEQUIEL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Datos
Se consumirán los datos del artículo de wikipedia sobre el deporte "Tennis" en inglés.

In [112]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Tren')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [113]:
# Demos un vistazo
article_text

'el tren es un vehículo compuesto por una serie de vagones o coches acoplados entre sí y remolcados por una locomotora,[1]\u200b o bien por coches autopropulsados que generalmente circulan sobre carriles permanentes. los trenes son usados para el transporte de mercancías, animales o personas de un lugar a otro. \nlos trenes tienen su origen en los vagones que eran impulsados por caballos o tirados por cables. tras la invención de la locomotora a vapor en el reino unido, los trenes se expandieron rápidamente, lo que permitió que las mercancías y los pasajeros se desplazaran de forma más rápida y económica ayudando a la industrialización y la interconexión. producto de la expansión de los trenes surgieron proyectos como el primer ferrocarril transcontinental de estados unidos, el ferrocarril transiberiano, el orient express o el ferrocarril de panamá entre mediados del siglo\xa0xix y principios del siglo\xa0xx. sin embargo, a partir de la década de 1920, producto de la invención del auto

In [114]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 19367


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [115]:
# Repaso de regex:
# https://docs.python.org/3/library/re.html

# Para practicar regex:
# https://regex101.com/

# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres: 
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

text = re.sub('\u200b', '', text) # substituir caracteres unicode

# probar en regex101 con los patrones anteriores:
# 'Hola [1], [], [ estoy bien   [123]. [12sss]. OK!   .'

In [116]:
# Demos un vistazo
text

'el tren es un vehículo compuesto por una serie de vagones o coches acoplados entre sí y remolcados por una locomotora,  o bien por coches autopropulsados que generalmente circulan sobre carriles permanentes. los trenes son usados para el transporte de mercancías, animales o personas de un lugar a otro. los trenes tienen su origen en los vagones que eran impulsados por caballos o tirados por cables. tras la invención de la locomotora a vapor en el reino unido, los trenes se expandieron rápidamente, lo que permitió que las mercancías y los pasajeros se desplazaran de forma más rápida y económica ayudando a la industrialización y la interconexión. producto de la expansión de los trenes surgieron proyectos como el primer ferrocarril transcontinental de estados unidos, el ferrocarril transiberiano, el orient express o el ferrocarril de panamá entre mediados del siglo xix y principios del siglo xx. sin embargo, a partir de la década de 1920, producto de la invención del automóvil, el uso de

In [117]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 19331


### 3 - Dividir el texto en sentencias y en palabras

In [118]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [119]:
# Demos un vistazo
corpus[:10]

['el tren es un vehículo compuesto por una serie de vagones o coches acoplados entre sí y remolcados por una locomotora,  o bien por coches autopropulsados que generalmente circulan sobre carriles permanentes.',
 'los trenes son usados para el transporte de mercancías, animales o personas de un lugar a otro.',
 'los trenes tienen su origen en los vagones que eran impulsados por caballos o tirados por cables.',
 'tras la invención de la locomotora a vapor en el reino unido, los trenes se expandieron rápidamente, lo que permitió que las mercancías y los pasajeros se desplazaran de forma más rápida y económica ayudando a la industrialización y la interconexión.',
 'producto de la expansión de los trenes surgieron proyectos como el primer ferrocarril transcontinental de estados unidos, el ferrocarril transiberiano, el orient express o el ferrocarril de panamá entre mediados del siglo xix y principios del siglo xx.',
 'sin embargo, a partir de la década de 1920, producto de la invención del

In [120]:
# Demos un vistazo
words[:20]

['el',
 'tren',
 'es',
 'un',
 'vehículo',
 'compuesto',
 'por',
 'una',
 'serie',
 'de',
 'vagones',
 'o',
 'coches',
 'acoplados',
 'entre',
 'sí',
 'y',
 'remolcados',
 'por',
 'una']

In [121]:
print("Vocabulario:", len(words))

Vocabulario: 3482


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [122]:
# Importo un lematizador en español
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SpanishStemmer

spanish_stemmer = SpanishStemmer()

lemmatizer = WordNetLemmatizer()

def perform_stem(tokens):
    return [spanish_stemmer.stem(token) for token in tokens]

def perform_lematization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_stem(nltk.word_tokenize(document.lower().translate(punctuation_removal), language='spanish'))

In [123]:
# Probar funcion get_processed_text
get_processed_text("compre, comprar, compramos, compran")

['compr', 'compr', 'compr', 'compr']

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"

    # Tomar las spanish_stopwords de la URL https://github.com/Alir3z4/stop-words/blob/master/spanish.txt
    spanish_stopwords = urllib.request.urlopen('https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt')
    # Cpmvertir a lista
    spanish_stopwords = [line.decode("utf-8").strip() for line in spanish_stopwords.readlines()]
    
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=spanish_stopwords)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "Perdón, no entiendo lo que dices"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias a ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [125]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!pip install gradio --quiet

In [126]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

C:\Users\EZEQUIEL\AppData\Local\Temp\ipykernel_20260\1968786774.py:9: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Q: Que motor tienen los trenes


d:\Coqui\UBA\Especializacion en Inteligencia Artificial\Procesamiento del Lenguaje Natural\procesamiento_lenguaje_natural (MIO)\venv_PLN\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\Coqui\UBA\Especializacion en Inteligencia Artificial\Procesamiento del Lenguaje Natural\procesamiento_lenguaje_natural (MIO)\venv_PLN\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['actual', 'adel', 'ademas', 'afirm', 'agreg', 'ahi', 'ahor', 'algun', 'ample', 'apen', 'aproxim', 'aqu', 'aquell', 'arrib', 'asegur', 'asi', 'aun', 'aunqu', 'añad', 'baj', 'bastant', 'cad', 'cerc', 'ciert', 'cinc', 'com', 'coment', 'conoc', 'consegu', 'consider', 'consig', 'contr', 'cos', 'cre', 'cualqui', 'cuand', 'cuant', 'cuatr', 'cuent', 'dad', 'deb', 'dec', 'dej', 'demas', 'de

A: los trenes actuales suelen ir propulsados por motores eléctricos, por sistemas diésel-eléctricos o hidráulicos, o por motor diésel directo.
Q: Que tipos de carga transportan los trenes
A: hay muchos tipos diferentes de trenes de carga, que se utilizan para transportar una gran variedad de diferentes tipos de carga, con varios tipos de vagones.
Q: Que es un tren ligero?
A: el tren ligero es un tren de la familia de los tranvías, en ciertos casos de piso alto con estaciones con plataformas, que circula en segmentos parcial o totalmente segregados del tránsito vehicular, con carriles reservados, vías apartadas y en algunos casos por túneles o en la superficie del centro de la ciudad.
Q: Cuales son las propuetas del tren para el futuro
A: 50 veces menos co2 que los aviones y las propuestas "futuristas" (que es posible realizar en un futuro próximo) incluyen trenes cuyas locomotoras son movidas por potentes motores de célula de hidrógeno en lugar de los conocidos motores diésel.
Q: Juega

# ACLARACIONES

Parece ser que para el Ingles las librerias proponen soluciones pre configuradas, mientras que para el Español hay que hacer algunos ajustes manualmente.

No encontré un lematizador en español dentro de la libreria nltk, por lo que utilicé un steammer.

Utilicé las stop_words en español del siguiente repositorio: https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt

Continué usando el vectorizador TFIDF.

# CONCLUSIONES

Acorde a lo visto en clase, podemos observar que el modelo intenta encontrar un documento dentro del corpus que tenga una similaridad coseno con el texto ingresado por el usuario. En los ejemplos que se enviaron como consulta se puede observar como responde con frases relativamente correctas cuando se le pregunta por el motor o por los tipos de carga, pero el problema mas grave es que a veces responde con algun documento que no se ajusta a lo consultado por el usuario, solo por el hecho de que coinciden algunas palabras en el documento, aunque se encuentren fuera de contexto.
Se realiza la prueba de ingresar palabras que supuestamente no deberían pertenecer al corpus, como el caso de la frase "Juega Futbol?" en el contexto de un articulo de trenes, y el modelo responde la frase pre armada para cuando todos los documentos del corpus tienen similaridad coseno igual a 0, lo cual es correcto.
En definitiva, es un modelo muy basico que puede servir para buscar alguna informacion precisa en algun documento extenso ingresando parte de las palabras del documento, sabiendo que esa informacion debería estar ahi. Porque si uno busca informacion sin saber que hay en el corpus es muy dificil lograr que nos devuelva una respuesta acorde a lo que se consulta.
Se podría mejorar la respuesta del modelo si permitieramos al usuario visualizar los otros documentos que tienen similaridad coseno alta en referencia a lo que se preguntó. 